In [3]:
import json
import requests

In [5]:
API_URL = "https://maayanlab.cloud/sigcom-lincs/metadata-api/signatures/value_count"

payload = {
    "where": {
        "meta.pert_name":  "dexamethasone"
    }
}

res = requests.get(API_URL, params={"filter": json.dumps(payload)})
results = res.json()

print(json.dumps(results, indent=2))

{
  "meta.data_level": {
    "5": 468
  },
  "meta.version": {
    "1": 468
  },
  "meta.pert_name": {
    "dexamethasone": 468
  },
  "meta.pert_type": {
    "Chemical": 468
  },
  "meta.$validator": {
    "https://raw.githubusercontent.com/MaayanLab/sigcom-lincs/main/validators/l1000_signatures.json": 468
  },
  "library": {
    "54198d6e-fe17-5ef8-91ac-02b425761653": 467,
    "42cd56da-0ad8-5dad-b27c-fe1d135401b2": 1
  },
  "meta.pubchem_id": {
    "5743": 467
  },
  "meta.pert_dose": {
    "10 uM": 248,
    "0.37 uM": 53,
    "1.11 uM": 21,
    "3.33 uM": 14,
    "0.01 uM": 13,
    "2.22 uM": 13,
    "0.74 uM": 13,
    "0.03 uM": 13,
    "0.25 uM": 13,
    "0.08 uM": 13,
    "0.125 uM": 12,
    "0.04 uM": 12,
    "20 uM": 9,
    "0.12 uM": 9,
    "1000 uM": 2,
    "30 uM": 2,
    "100 uM": 2,
    "320 uM": 2,
    "4 uM": 1,
    "8 uM": 1,
    "12.5 uM": 1
  },
  "meta.pert_time": {
    "24 h": 229,
    "6 h": 228,
    "48 h": 9,
    "72 h": 1
  },
  "meta.creation_time": {
    "202